# 目的：預計建立個指標結果


In [91]:
import os
import pandas as pd

def convert_minguo_to_gregorian(date_str):
    """
    將中華民國（民國）日期轉換為西元日期。
    
    參數:
        date_str (str): 民國日期字串，格式為 'YYY/MM/DD' 或 'YYY-MM-DD'。
        
    返回:
        str: 西元日期字串，格式為 'YYYY/MM/DD'。
    """
    if pd.isnull(date_str):
        return date_str
    try:
        parts = date_str.split('/')
        if len(parts) != 3:
            parts = date_str.split('-')
        year = int(parts[0]) + 1911
        return f"{year}/{parts[1]}/{parts[2]}"
    except Exception as e:
        print(f"Error converting date: {date_str} - {e}")
        return date_str


# 定義函數合併並檢查資料格式
def merge_and_check_format(folder_path, output_csv_path):
    """
    合併指定資料夾中的所有 CSV 檔案，並檢查合併後的 DataFrame 中每個欄位的資料格式。
    
    參數:
        folder_path (str): 包含 CSV 檔案的資料夾路徑。
        output_csv_path (str): 輸出合併後 CSV 檔案的路徑。
    """
    # 獲取資料夾下的所有檔案
    files = os.listdir(folder_path)
    dfs = []

    # 讀取每個檔案並合併到一個 DataFrame 中
    for file in files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path, encoding='utf-8')
        dfs.append(df)

    merged_df = pd.concat(dfs, ignore_index=True)

    # 將 '日期' 欄位從民國紀年轉換為西元紀年，並直接轉換為 Timestamp
    if '日期' in merged_df.columns:
        merged_df['日期'] = merged_df['日期'].apply(convert_minguo_to_gregorian)
        merged_df['日期'] = merged_df['日期'].apply(lambda x: pd.Timestamp(x) if pd.notnull(x) else pd.NaT)
        
        # 打印出缺失值所在的 loc 資料
        missing_dates = merged_df[merged_df['日期'].isnull()]
        if not missing_dates.empty:
            print("以下為缺失值所在的 loc 資料：")
            print(missing_dates)
        
        # 使用填充值填充缺失值
        merged_df['日期'].fillna(value=pd.Timestamp('1900-01-01'), inplace=True)
    
    merged_df['成交金額'] = round(merged_df['成交金額'].astype(float)).astype(float)

    # 檢查每個資料格的資料格式
    different_formats = {}

    for col in merged_df.columns:
        formats = merged_df[col].apply(type).unique()
        if len(formats) > 1:
            different_formats[col] = formats

    if different_formats:
        print("下列格子的資料格式不同：")
        for col, formats in different_formats.items():
            print(f"列名: {col}, 不同的資料格式: {formats}")
    else:
        print("所有格子的資料格式相同：")
        for col in merged_df.columns:
            print(f"列名: {col}, 資料格式: {merged_df[col].dtype}")

    # # 將合併後的 DataFrame 存儲為 CSV 檔案
    # merged_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    return merged_df

# 指定資料夾路徑和輸出 CSV 檔案路徑
folder_path = r"C:\Users\D\Desktop\project\股票程式碼\股票\2618_長榮航"
output_csv_path = "合併資料.csv"

# 呼叫函數合併並檢查資料格式，並將結果存儲為 CSV 檔案
merged_df = merge_and_check_format(folder_path, output_csv_path)

所有格子的資料格式相同：
列名: 日期, 資料格式: datetime64[ns]
列名: 成交股數, 資料格式: float64
列名: 成交金額, 資料格式: float64
列名: 開盤價, 資料格式: float64
列名: 最高價, 資料格式: float64
列名: 最低價, 資料格式: float64
列名: 收盤價, 資料格式: float64
列名: 漲跌價差, 資料格式: float64
列名: 成交筆數, 資料格式: float64


# Kd (隨機指標) 。 stochastic_indicator
說明：\
當%K和%D值都高於80時，表示市場可能已經超買，價格可能會下跌。\
當%K和%D值都低於20時，表示市場可能已經超賣，價格可能會上漲。\
算式：\
%K = [ (今日收盤價-過去N天最低價) / (過去N天最高價 - 過去N天最低價) ] * 100%\
%D = %K的M天移動平均\
備註：\
N取14、K取3

In [51]:
today_closing_price 
highest_price_past_n_days
lowest_price_past_n_days
k_moving_average_of_n_days


stochastic_indicator_K , stochastic_indicator_D = 21,81
if (stochastic_indicator_K > 80) and (stochastic_indicator_D > 80):
    print("隨機指標 超買中，找時間賣出")
elif (stochastic_indicator_K < 20) and (stochastic_indicator_D < 20):
    print("隨機指標 超賣中，找時間買入")
else:
    print("隨機指標 不明顯")

In [84]:
print(type(merged_df.loc[2]["日期"] ))
delta = merged_df.loc[2]["日期"] - merged_df.loc[1]["日期"] 
# 從 Timedelta 對象中提取天數
days = delta.days

# 打印天數
print(type(days))
print(days)

delta = merged_df.loc[2]["日期"] - merged_df.loc[1]["日期"] 
# 從 Timedelta 對象中提取天數
days = (delta + timedelta(days=1)).days
print(days)

today = datetime.today().date()
today_datetime64 = pd.to_datetime(today)

print(type(today_datetime64))
print(today_datetime64)

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'int'>
1
2
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2024-05-21 00:00:00


In [42]:
from datetime import timedelta , datetime

test = merged_df.loc[1]["日期"] 
# 將日期增加一天
test = test + timedelta(days=1)
test

Timestamp('2010-01-06 00:00:00')

In [ ]:
import pandas as pd

def get_closing_price(df, date_str, flag='N'):
    """
    根據給定的日期字符串，回傳該日期或最近日期的收盤價。
    
    參數：
    - df：包含日期和價格資料的數據框。
    - date_str：日期字符串。
    - flag：標示是否需要回傳最接近日期的價格，Y表示是，N表示否。
    
    回傳：
    - 若flag為Y，回傳最近日期及其收盤價。
    - 若flag為N，回傳該日期的收盤價。
    """
    target_date = pd.to_datetime(date_str).astype('datetime64[ns]')
 
    if flag == 'Y' and target_date not in df["日期"].values:
        closest_index = df["日期"].searchsorted(target_date) - 1
        closest_date = df.loc[closest_index]["日期"]
        closing_price = df.loc[closest_index]["收盤價"]
        return closest_date, closing_price
    else:
        closing_price = df[df["日期"] == target_date]["收盤價"].values[0]
        print("else")
        return closing_price

# 假設數據框為 merged_df，調用方式如下：
result = get_closing_price(merged_df, '2024/05/2', 'Y')
print(result)


In [83]:
import pandas as pd
import numpy as np

# Define the date string
date_string = "2024/05/2"

# Convert to pandas Timestamp
timestamp = pd.to_datetime(date_string)

# Convert to numpy datetime64[ns]
date_datetime64 = np.datetime64(timestamp)

# Display the result and type to confirm the format
print(date_datetime64)
print(type(date_datetime64))


2024-05-02T00:00:00.000000
<class 'numpy.datetime64'>


In [74]:
import pandas as pd

# 定義兩個日期字符串
date_str1 = '2024/05/01'
date_str2 = '2024/05/19'

# 將日期字符串轉換為 Timestamp 格式
timestamp1 = pd.Timestamp(date_str1)
timestamp2 = pd.Timestamp(date_str2)

# 計算兩個日期之間的差異
date_diff = timestamp2 - timestamp1

# 打印結果
print(f"Date 1: {timestamp1}")
print(f"Date 2: {timestamp2}")
print(f"Difference: {date_diff}")


Date 1: 2024-05-01 00:00:00
Date 2: 2024-05-19 00:00:00
Difference: 18 days 00:00:00


In [52]:
merged_df


,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,2015-01-05,30581007.0,6.925431e+08,22.50,23.00,22.30,23.00,0.85,8047.0
1,2015-01-06,50767924.0,1.182647e+09,23.05,23.55,22.95,22.95,-0.05,12277.0
2,2015-01-07,51527061.0,1.216460e+09,23.40,23.90,23.20,23.50,0.55,11343.0
3,2015-01-08,48550596.0,1.147522e+09,23.90,23.95,23.05,23.60,0.10,9587.0
4,2015-01-09,48581709.0,1.169014e+09,23.60,24.50,23.45,24.50,0.90,10122.0
...,...,...,...,...,...,...,...,...,...
2282,2024-05-15,88483708.0,3.203988e+09,36.70,36.75,35.85,36.00,-0.40,31062.0
2283,2024-05-16,134797823.0,4.959238e+09,36.25,37.20,35.90,37.00,1.00,38323.0
2284,2024-05-17,145408649.0,5.447180e+09,37.50,37.95,37.05,37.40,0.40,43711.0
2285,2024-05-20,238615417.0,8.786531e+09,37.70,37.80,36.10,36.40,-1.00,78748.0
